# Enhanced Candlestick Chart with Plotly

This notebook demonstrates how to create an enhanced candlestick chart using the improved Chart class, random OHLCV data, and technical indicators.

In [1]:
# Import necessary libraries
import pandas as pd
from data.random_data import RandomOHLCV
from frame.frame import Frame
from chart.chart import Chart

In [2]:
# Generate random data
data = RandomOHLCV(
    freq='5 min',
    head_max=0.3,
    tail_max=0.3,
    start='2023-01-01',
    open_val=100.00,
    periods=300,
    open_rng=(-0.4, 0.4),
    close_rng=(-0.4, 0.4),
    vol_rng=(-50, 60),
    volatility_rng=(0, 0.02),
    volatility_dur=3,
    volatility_freq=50
)

df = data.get_dataframe()

c:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\random_data.py:87: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  self.data = RandDataStore(pd.date_range(start=self.start, periods=self.periods, freq=self.freq))


In [3]:
from data.data_manager import DataManager
from strategies.ta import MA, MACD

dm = DataManager(df)
dm.add_ta( MA('close', 20))
dm.add_ta( MA('close', 50))
dm.add_ta( MACD('close', 12, 26, 9))
dm.data

,open,high,low,close,volume,MA_cl_20,MA_cl_50,MACD_cl_12_26_9_MACD,MACD_cl_12_26_9_Signal,MACD_cl_12_26_9_Histogram
date,,,,,,,,,,
2023-01-01 00:00:00,100.16,100.52,100.12,100.40,17639,NaN,NaN,0.000000,0.000000,0.000000
2023-01-01 00:05:00,100.12,100.39,100.09,100.15,500,NaN,NaN,-0.019943,-0.003989,-0.015954
2023-01-01 00:10:00,99.87,100.16,99.72,100.04,6271,NaN,NaN,-0.044116,-0.012014,-0.032102
2023-01-01 00:15:00,99.90,100.18,99.81,99.90,500,NaN,NaN,-0.073719,-0.024355,-0.049364
2023-01-01 00:20:00,100.12,100.35,99.80,100.03,500,NaN,NaN,-0.085703,-0.036625,-0.049078
...,...,...,...,...,...,...,...,...,...,...
2023-01-02 00:35:00,94.66,94.79,94.29,94.36,22894,94.8735,94.3266,-0.006300,0.065834,-0.072134
2023-01-02 00:40:00,94.06,94.56,93.79,94.36,500,94.8730,94.3212,-0.032307,0.046206,-0.078513
2023-01-02 00:45:00,94.34,94.60,94.12,94.51,9211,94.8820,94.3202,-0.040349,0.028895,-0.069243


In [4]:
# Create and display the enhanced chart
chart = Chart( title='Enhanced Random Data Chart', rowHeights=[0.2, 0.1, 0.7], height=800, width=1200)
chart.add_candles_and_volume(df)

# Add trading hours (assuming market hours are 9:30 AM to 4:00 PM)
chart.add_trading_hours(df, [('09:30', '16:00')])
from strategies.ta import MA, MACD

chart.refesh(df)
chart.add_ta(MA('close', 9).run(df), {'color': 'blue', 'width': 1}, 'line')
chart.add_ta(MA('close', 21).run(df), {'color': 'red', 'width': 1}, 'line')
chart.add_ta(MACD('close', 12, 26, 9).run(df), [
    {'color': 'green', 'width': 1},
    {'color': 'red', 'width': 1},
    {'color': 'blue', 'width': 1}], 'macd')

chart.show(width=1400)

In [5]:
# MACD('close', 12, 26, 9).run(df)

In [6]:
from dataclasses import dataclass, field
from typing import Any, Dict, List, Tuple
from data.data_manager import DataManager
from chart.chart import Chart
from strategies.ta import Indicator, MA, MACD, HPLP, SupportResistance

@dataclass
class Frame:
    data: pd.DataFrame
    symbol: str
    trading_hours: List[Tuple[str, str]]
    indicators: List[Tuple[Indicator, Dict[str, Any]]] = field(default_factory=list)

    def __post_init__(self):
        self.dm = DataManager(self.data)
        self.chart = Chart(title=self.symbol, rowHeights=[0.2, 0.1, 0.7], height=800, width=800)
        self.chart.add_candles_and_volume(self.dm.data)
        self.chart.add_trading_hours(self.dm.data, self.trading_hours)

    def add_ta(self,  indicator: Indicator, style: Dict[str, Any], row: int = 1):
        self.dm.add_ta(indicator)
        self.indicators.append((indicator, style, row))

    # def update_data(self, new_data: Any):
    #     self.dm.update_data(new_data)
    #     self._update_chart()

    def _update_chart(self):
        for indicator, style, row in self.indicators:
            chart_type = "line"
            if indicator.__class__.__name__ in ["MACD"]: chart_type = "macd"
            if indicator.__class__.__name__ in ["HPLP"]: chart_type = "points"
            self.chart.add_ta(self.dm.data[indicator.names], style, chart_type, row)


    def plot(self, width:int=None, height:int=None):
        self._update_chart()
        self.chart.show(width=width, height=height)

    def plot_refresh(self):
        self.chart.refesh()
        self._update_chart()
        self.chart.show()

f5min = Frame(df, '5min', [('09:30', '16:00')])
f5min.add_ta(MA('close', 9), {'color': 'blue', 'width': 1})
f5min.add_ta(MA('close', 21), {'color': 'red', 'width': 1})
f5min.add_ta(MACD('close', 12, 26, 9), [
    {'color': 'green', 'width': 1},
    {'color': 'red', 'width': 1},
    {'color': 'blue', 'width': 1}], row=3)
f5min.add_ta(HPLP(hi_col='high', lo_col='low', span=10), [
    {'color': 'green', 'size': 10, 'opacity': 0.99},
    {'color': 'red', 'size': 10, 'opacity': 0.99}], row=1)
# f5min.add_ta(SupportResistance(hi_col='high', lo_col='low', tolerance=0.01, touch_tolerance=0.005),
#              [{'color': 'purple', 'width': 1}, {'color': 'orange', 'width': 1}],
#              chart_type='support_resistance', row=1)
f5min.plot(width=2000, height=1000)
f5min.plot(width=2000 )

In [7]:
from strategies.ta import MA, MACD, HPLP, SupportResistance
from frame.frame import Frame

# Make sure to restart the kernel and run all cells from the beginning to ensure you're using the latest version of the Frame class

f5min = Frame(df, '5min', [('09:30', '16:00')])
f5min.add_ta(MA('close', 9), {'color': 'blue', 'width': 1})
f5min.add_ta(MA('close', 21), {'color': 'red', 'width': 1})
f5min.add_ta(MACD('close', 12, 26, 9), [
    {'color': 'green', 'width': 1},
    {'color': 'red', 'width': 1},
    {'color': 'blue', 'width': 1}], chart_type='macd', row=3)
f5min.add_ta(HPLP(hi_col='high', lo_col='low', span=10), [
    {'color': 'green', 'size': 10, 'opacity': 0.99},
    {'color': 'red', 'size': 10, 'opacity': 0.99}], chart_type='points', row=1)
f5min.add_ta(SupportResistance(hi_col='HP_hi_10', lo_col='LP_lo_10', tolerance=0.01, touch_tolerance=0.005),
             [{'color': 'purple', 'width': 1}, {'color': 'orange', 'width': 1}],
             chart_type='support_resistance', row=1)
f5min.plot(width=2000, height=1000)

Recent support level: [nan]
Recent resistance level: [nan, nan, nan, nan, nan, nan, nan]
Finished adding support and resistance levels


In [9]:
import numpy as np

def add_support_resistance(self, data, style):
    support_cols = [col for col in data.columns if col.endswith('Support')]
    resistance_cols = [col for col in data.columns if col.endswith('Resistance')]
    
    if not support_cols or not resistance_cols:
        raise ValueError("Support or Resistance columns not found in the data")

    support_col = support_cols[0]
    resistance_col = resistance_cols[0]
    support_band_col = 'Support_Band'
    resistance_band_col = 'Resistance_Band'

    # Extract the most recent support and resistance levels
    recent_support = data[support_col].iloc[-1]
    recent_resistance = data[resistance_col].iloc[-1]
    support_band = data[support_band_col].iloc[-1]
    resistance_band = data[resistance_band_col].iloc[-1]

    print("Recent support level:", recent_support)
    print("Recent resistance level:", recent_resistance)
    print("Support band levels:", support_band)
    print("Resistance band levels:", resistance_band)

    # Add the most recent support and resistance levels as lines
    self.fig.add_shape(
        type="line",
        x0=data.index[0], x1=data.index[-1], y0=recent_support, y1=recent_support,
        line=dict(color=style[0]['color'], width=style[0]['width'], dash="dash"),
        row=1, col=1
    )
    self.fig.add_shape(
        type="line",
        x0=data.index[0], x1=data.index[-1], y0=recent_resistance, y1=recent_resistance,
        line=dict(color=style[1]['color'], width=style[1]['width'], dash="dash"),
        row=1, col=1
    )

    # Add shaded areas for the support and resistance bands
    self.fig.add_shape(
        type="rect",
        x0=data.index[0], x1=data.index[-1], y0=min(support_band), y1=max(support_band),
        fillcolor=style[0]['color'], opacity=0.2, line_width=0,
        row=1, col=1
    )
    self.fig.add_shape(
        type="rect",
        x0=data.index[0], x1=data.index[-1], y0=min(resistance_band), y1=max(resistance_band),
        fillcolor=style[1]['color'], opacity=0.2, line_width=0,
        row=1, col=1
    )

    print("Finished adding support and resistance levels")

add_support_resistance(f5min, df, [{'color': 'purple', 'width': 1}, {'color': 'orange', 'width': 1}])
f5min.plot(width=2000, height=1000)

ValueError: Support or Resistance columns not found in the data

In [ ]:
f5min.dm.data

,open,high,low,close,volume,MA_cl_9,MA_cl_21,MACD_cl_12_26_9_MACD,MACD_cl_12_26_9_Signal,MACD_cl_12_26_9_Histogram,HP_hi_10,LP_lo_10,SR_Resistance,SR_Support,SR_Resistance_Strength,SR_Support_Strength,Recent_Support,Recent_Resistance,Support_Band,Resistance_Band
date,,,,,,,,,,,,,,,,,,,,
2023-01-01 00:00:00,99.67,99.72,99.30,99.47,26531,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,"[98.56133333333334, 99.49438202247191, 100.415...","[98.10514285714285, 98.98714285714286, 99.8692...","[35, 87, 56, 31, 42, 40, 15, 15]","[39, 75, 61, 31, 36, 41, 13, 14]",105.154615,105.694615,[105.15461538461538],[105.69461538461539]
2023-01-01 00:05:00,99.46,99.84,99.41,99.78,500,NaN,NaN,0.024729,0.004946,0.019783,NaN,NaN,"[98.56133333333334, 99.49438202247191, 100.415...","[98.10514285714285, 98.98714285714286, 99.8692...","[35, 87, 56, 31, 42, 40, 15, 15]","[39, 75, 61, 31, 36, 41, 13, 14]",105.154615,105.694615,[105.15461538461538],[105.69461538461539]
2023-01-01 00:10:00,99.44,99.87,99.41,99.67,500,NaN,NaN,0.035047,0.010966,0.024081,NaN,NaN,"[98.56133333333334, 99.49438202247191, 100.415...","[98.10514285714285, 98.98714285714286, 99.8692...","[35, 87, 56, 31, 42, 40, 15, 15]","[39, 75, 61, 31, 36, 41, 13, 14]",105.154615,105.694615,[105.15461538461538],[105.69461538461539]
2023-01-01 00:15:00,100.00,100.16,99.58,99.67,29469,NaN,NaN,0.042732,0.017319,0.025413,100.16,NaN,"[98.56133333333334, 99.49438202247191, 100.415...","[98.10514285714285, 98.98714285714286, 99.8692...","[35, 87, 56, 31, 42, 40, 15, 15]","[39, 75, 61, 31, 36, 41, 13, 14]",105.154615,105.694615,[105.15461538461538],[105.69461538461539]
2023-01-01 00:20:00,99.82,100.02,99.61,99.83,500,NaN,NaN,0.061029,0.026061,0.034968,NaN,NaN,"[98.56133333333334, 99.49438202247191, 100.415...","[98.10514285714285, 98.98714285714286, 99.8692...","[35, 87, 56, 31, 42, 40, 15, 15]","[39, 75, 61, 31, 36, 41, 13, 14]",105.154615,105.694615,[105.15461538461538],[105.69461538461539]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-02 00:35:00,104.07,104.33,103.50,103.70,500,103.401111,102.849048,0.302360,0.171399,0.130962,NaN,NaN,"[98.56133333333334, 99.49438202247191, 100.415...","[98.10514285714285, 98.98714285714286, 99.8692...","[35, 87, 56, 31, 42, 40, 15, 15]","[39, 75, 61, 31, 36, 41, 13, 14]",105.154615,105.694615,[105.15461538461538],[105.69461538461539]
2023-01-02 00:40:00,103.89,103.99,103.45,103.65,500,103.522222,102.942381,0.303060,0.197731,0.105329,NaN,NaN,"[98.56133333333334, 99.49438202247191, 100.415...","[98.10514285714285, 98.98714285714286, 99.8692...","[35, 87, 56, 31, 42, 40, 15, 15]","[39, 75, 61, 31, 36, 41, 13, 14]",105.154615,105.694615,[105.15461538461538],[105.69461538461539]
2023-01-02 00:45:00,103.84,104.47,103.55,104.23,23477,103.626667,103.070000,0.346422,0.227469,0.118953,104.47,NaN,"[98.56133333333334, 99.49438202247191, 100.415...","[98.10514285714285, 98.98714285714286, 99.8692...","[35, 87, 56, 31, 42, 40, 15, 15]","[39, 75, 61, 31, 36, 41, 13, 14]",105.154615,105.694615,[105.15461538461538],[105.69461538461539]
